### Detector Systematics 
- Load the DetVar samples, applies the nue selection on them
- The after selection pickl is used to study the variations

-- efficieny at different stages

-- in function of a variable (plot)

In [22]:
import nue_selection_helper as nue_helper
import glob
import pandas as pd
import numpy as np

In [4]:
write_pickle = False
outfile = './input/Jun2020/lite/sys_after_training.pckl'
input_dir = "./input/Jun2020/syst/"
output_dir = "./output/"

In [6]:
samples = ['_'.join(f.split('/')[-1].split('.')[0].split('_')[:-1]) for f in glob.glob(input_dir+'*.pckl')]

In [8]:
if write_pickle:
    nue_helper.CreateAfterTraining(samples, input_dir, one_file=outfile)
data = pd.read_pickle(outfile)

In [9]:
data.keys()

dict_keys(['nue_DetVar_WireModAngleYZ', 'nue_DetVar_LYRayleigh', 'nue_DetVar_LYAttenuation', 'nue_DetVar_wiremod_ScaleYZ', 'nue_DetVar_LYDown', 'nue_DetVar_wiremod_ScaledEdX', 'nue_DetVar_WireModAngleXZ', 'nue_DetVar_CV', 'nue_DetVar_wiremod_ScaleX'])

In [27]:
# We grouped by 'event' was this a smart choice?
assert len(np.unique(data['nue_DetVar_WireModAngleYZ']['daughters'].xs(0, level='daughter').index)) == len(data['nue_DetVar_WireModAngleYZ']['daughters'].xs(0, level='daughter').index)